In [2]:
search_for = 132
start_from = 0
threads = 10
thread = 0
log_freq = 10000



#in this cell, for a given lattice, we compute all forbidden nodes, 
#i.e. those nodes whose Voronoi cell is at a distance less than doubled covering radius

#An* represented in m=n+1 as sum x_i=0 with x_i\equiv to each other mod m. 
#Forbidden nodes will be recorded w.r.t the basis (-n,1,1,..) and permutations

n=5 #lattice dim
m=n+1 #space dim




R = sqrt(n*(n+1)*(n+2)/12) #covering radius

#first we generate all nodes with norm <=4R, and with norm<=2R
t=1 #.3 #can increase a bit for random from non-forbidden
short = []
vshort = []
K = ceil(t*4*R)
K2 = (t*4*R)^2
Kv = (2*R)^2

indexes = [] #arrays of integers between -K and K having the same residue modulo m 
for rem in range(m):
    res = []
    for i in range(-K,K+1):
        if i%m==rem:
            res.append(i)
    indexes.append(res)

for c in range(0,K+1):
    for coefs in mrange_iter([indexes[c%m] for i in range(m-2)]):
        p = [c]+list(coefs)+[-c-sum(coefs)]
        is_sorted = True
        for i in range(m-1):
            if p[i]<p[i+1]:
                is_sorted = False
                break
        if is_sorted:
            norm2 = sum((p[i])^2 for i in range(m))
            if norm2>0:
                if norm2 <= Kv:
                    vshort.append(p)
                if norm2 <= K2:
                    short.append(p)           

print("very short:",len(vshort))
print("short:",len(short))


from time import time
tm = time()


#defining the cone arising due to symmetry - it suffices to construct Voronoi cell in it
ineqs = [] 
for i in range(m-1):
    toadd = [0]*(m+1)
    toadd[i+1]=1
    toadd[i+2]=-1
    ineqs.append(toadd) 
    
ineqs += [[0]+[1]*m,[0]+[-1]*m] #we are in the hyperplane where all coords sum to zero

#now Voronoi cell planes
for p in vshort: #suffices to look at nodes at most 2R
    ineqs.append([sum((p[i])^2 for i in range(m))]+[-2*p[i] for i in range(m)])

V = Polyhedron(ieqs=ineqs)
    
print(time()-tm)    

### orthogonal projection affine operator onto (improper) non-empty polytope 
zerom = matrix(m)
def projp(P):
    nv = P.n_vertices()
    z = vector(P.vertices()[0])
    if nv==1:
        return zerom,z
    A = matrix([vector(P.vertices()[i])-z for i in range(1,nv)]).transpose()
    return A*A.pseudoinverse(),z


forb = []

for d in range(0,n): #dim of face
    for ff in V.faces(d):
        proj = projp(ff)
        for p in short:
            if p not in forb:
                x = vector([t/2 for t in p]) #dist from 1/2 of a node to the Voronoi cell equals 1/2 of the dist between cells at zero and at the node
                proj_pnt = proj[0]*(x-proj[1])+proj[1]
                if proj_pnt in ff.as_polyhedron():
                    dist = (x-proj_pnt).norm()
                    if dist<R:
                        forb.append(p)

print("forbidden:",len(forb))

#expand by the basis (-n,1,1,..) and permutations
def basis_coefs(x):
    y = x[0:(m-1)]
    sy = sum(y)
    return [(-sy-y[i])/m for i in range(m-1)]

### forbd = forb with possible permutations
forbd = {}
forbdv = []
for x in forb:
    for t in Permutations(list(x)):
        c = vector(basis_coefs(t))
        if str(c) not in forbd:
            forbd[str(c)]=0
            if sum(c)>=0:
                forbdv.append(c)
print("forbidden without symmetries:",len(forbd))
###





fn = 0
for x in forb:
    norm2 = sum(t^2 for t in x)
    if norm2>fn:
        fn = norm2
print("square L2 norm of forbidden:",fn)


small = 2 #max abs value of a small coefficient
sc = []
for c in mrange_iter([range(-small,small+1) for i in range(n)]):
    if sum(c)>=0:
        sc.append(vector(c))
sc.sort(key=lambda i: sum(x*x for x in i)) #abs(x) in place of x*x seems a bit slower
sc = sc[1:] #removing zero

print("small:",len(sc))

order = [(-1)^i*(1/4+i/2)-1/4 for i in range(1000)] #[0,-1,1,-2,2,...]

def check_sublattice(basis,target=Infinity): 
    A = matrix(basis) #rows
    #A = A.LLL() #benchmark to see if helps
    A = A.transpose() #columns

#     Ad = abs(A.det())
#     if Ad==0 or Ad>=target:
#         return Infinity
        
    for coefs in sc:
        if str(A*coefs) in forbd:
            return Infinity

    Ai = A.inverse()
    Ai_norms = [sum(t^2 for t in Ai[i]) for i in range(n)]
    M = []
    Mp = 1
    Mps = 1
    for i in range(n):
        mi = floor(sqrt(fn*Ai_norms[i]))
        M.append(mi)
        Mp *= (2*mi+1)
        Mps *= (2*min(small,mi)+1)
        
    if max(M)>small: #otherwise already considered
        if Mp-Mps<len(forbd): #faster using A, otherwise A_inv
            for coefs in mrange_iter([order[0:(2*M[i]+1)] for i in range(n)]):
                if sum(coefs)>=0 and max(abs(x) for x in coefs)>small:
                    if str(A*vector(coefs)) in forbd:
                        return Infinity
        else: #check if A_inv times forbidden is not in Z^n
            for x in forbdv:
                y = Ai*x
                allint = True
                for t in y:
                    if t!=floor(t):
                        allint = False
                        break
                if allint:
                    return Infinity   
    
    return 0#Ad



#random/exhaustive sublattice search, skip to the cell starting with good for search using non-forbidden generating vectors of small norm
#each type of a matrix is the numbers on the diagonal
maxd = search_for #bound for diagonal entries
mindet = search_for #minimum determinant
maxdet = search_for #maximum determinant

def all_types(m,d1,d2,n):
    res = []
    if n==1:
        for k in range(d1,min(d2,m)+1):
            res.append([k])
        return res
    for k in range(1,min(d2,m)+1):
        prev = all_types(m,ceil(d1/k),floor(d2/k),n-1)
        for x in prev:
            res.append([k]+x)
    return res

types = all_types(maxd,mindet,maxdet,n)

print("types:",len(types))

def ind_max(t): #given type, lists maxima of indexes to define entries above the diagonal
    return [t[i]^i for i in range(1,n)]


allc = 0
for t in types:
    allc += prod(ind_max(t))
print("all cases:",allc)

from random import randint
def rand_ind(t): #given type, return random indexes
    maxs = ind_max(t)
    return [randint(0,maxs[i]-1) for i in range(n-1)]

def sublat_mat(t,ind): #given type and indexes, return sublattice basis
    res = matrix(n)
    for i in range(n):
        res[i,i] = t[i]
    for i in range(1,n):
        tt = -floor(t[i]/2)
        indc = ind[i-1]
        for j in range(i):
            res[j,i] = tt + (indc % t[i])
            indc = floor(indc / t[i])
    return res

def exh_search():
    tm = time()
    bestv = Infinity
    best = []
    allcnt = 0
    for tt in types:
        print("type:",tt,prod(tt))
        cnt = 0
        maxinds = ind_max(tt)
        cM = prod(maxinds)
        print("cases of this type:",cM)
        for i in range(cM):
            allcnt += 1
            if allcnt%(log_freq*threads)==0:
                print(allcnt,(time()-tm)/3600*(allc-allcnt)/(log_freq*threads))
                tm=time()
            if allcnt>=start_from and allcnt%threads==thread:
                inds = []
                l = i
                for j in range(n-1):
                    inds.append(l%maxinds[j])
                    l=floor(l/maxinds[j])
                mat = sublat_mat(tt,inds)
                cs = check_sublattice(mat)
                if cs==0:
                    cs = prod(tt)
                    if cs<bestv:
                        print("sublattice basis:")
                        print(mat)
                        bestv = cs
                        best = [mat]
                    elif cs==bestv:
                        best.append(mat)
        print("best so far:",len(best))
    print("all cases:",allcnt)
    print("best:",len(best))
    return best





best = exh_search()




very short: 5
short: 39
0.004119157791137695
forbidden: 31
forbidden without symmetries: 1984
square L2 norm of forbidden: 264
small: 1752
types: 375
all cases: 1268606955
type: [1, 1, 1, 1, 132] 132
cases of this type: 303595776
100000 64.51492081765377
200000 105.47886554390965
300000 101.62301759325246
400000 100.79386903307395
500000 100.41264780481464
600000 104.23908670308809
700000 101.63584882820943
800000 98.77009267449125
900000 98.81497073988399
1000000 99.30146436195756
1100000 97.73292723522847
1200000 26.070020525890115
1300000 96.134092212079
1400000 97.69740563771283
1500000 95.48232510372932
1600000 96.69551368571149
1700000 96.82950668302819
1800000 102.91405172614388
1900000 97.56155754072707
2000000 98.11372212715766
2100000 97.161687901691
2200000 98.94684056953258
2300000 76.37164527635284
2400000 46.58560888983651
2500000 93.79126499893768
2600000 93.60005026329534
2700000 92.06481033897401
2800000 92.55409686619444
2900000 95.5337933308718
3000000 94.49499020286

29600000 100.7118981526435
29700000 68.13266856310088
29800000 75.1703285237208
29900000 105.94606064514188
30000000 98.82398667353911
30100000 97.34800251754618
30200000 41.29828775520203
30300000 99.66630663727794
30400000 100.64320593439155
30500000 97.72183429395555
30600000 96.01567273308771
30700000 97.10608060296623
30800000 95.02659445424456
30900000 92.42306969446588
31000000 89.8621247651425
31100000 26.2964363105516
31200000 94.33059317525483
31300000 91.49789296830777
31400000 92.80796846593051
31500000 96.54071480944587
31600000 97.14743081672418
31700000 95.8792269378311
31800000 98.25869023230018
31900000 97.32958274264476
32000000 55.2376230019964
32100000 84.84245726470321
32200000 102.4157002026518
32300000 103.55206420057738
32400000 105.23769974542861
32500000 33.67283074287227
32600000 106.54068833984545
32700000 104.04879873370092
32800000 101.3508694737331
32900000 103.31777630077485
33000000 102.1473633063913
33100000 98.14970061756253
33200000 96.03035898838682

60000000 98.72693051648861
60100000 102.26279355947204
60200000 101.06655108135034
60300000 36.504550703957605
60400000 98.10915368649704
60500000 96.0377297120901
60600000 95.75024290350358
60700000 91.77513495115564
60800000 90.40023223487105
60900000 90.13793847024672
61000000 23.17491099099956
61100000 94.86483402237363
61200000 91.2342901712798
61300000 90.33759137084732
61400000 89.40988968360247
61500000 95.46785448159261
61600000 97.67904630264593
61700000 36.8757150559604
61800000 106.38823873739513
61900000 105.44563555609791
62000000 101.75809796463155
62100000 104.25854314365931
62200000 96.9487085298339
62300000 97.2412837501144
62400000 98.78240294610217
62500000 95.92770731197281
62600000 49.78578505247069
62700000 82.76771630711914
62800000 92.19313035585195
62900000 87.95109481537546
63000000 84.19739650534822
63100000 87.03018159368139
63200000 87.14066336261484
63300000 23.423310804409535
63400000 91.835118280929
63500000 87.28169235148255
63600000 84.00332804413492


90400000 44.75650438893521
90500000 56.68555295690377
90600000 75.12955652945246
90700000 71.58792846034865
90800000 78.48625307917018
90900000 17.454440169009388
91000000 80.13862447964533
91100000 73.01348743647216
91200000 79.04241917343529
91300000 69.154252586365
91400000 43.10425614849165
91500000 79.67933910271165
91600000 80.10302211262443
91700000 77.98904364664006
91800000 80.53015067734982
91900000 79.22962535740159
92000000 89.21706371333015
92100000 87.33002392744815
92200000 89.08399371508705
92300000 89.40953050987893
92400000 84.32163679324425
92500000 81.46109477459967
92600000 77.01970505125034
92700000 60.365264135999745
92800000 49.40068219879686
92900000 75.43671800936382
93000000 79.40851758320059
93100000 67.08524884885502
93200000 15.026530456579323
93300000 83.81823807891364
93400000 73.52685527314394
93500000 70.70380576843884
93600000 48.38826888017387
93700000 44.57177760704609
93800000 62.24997463926225
93900000 78.79102562147622
94000000 83.9230582180447
9

120000000 45.191899021825016
120100000 46.74484977805754
120200000 48.09844450661556
120300000 40.33757391052932
120400000 39.118968828531884
120500000 34.96523751525901
120600000 27.703456068514566
120700000 44.83286892937087
120800000 13.472426283803305
120900000 55.4051532778
121000000 36.605975814923724
121100000 40.873372376257784
121200000 44.53079647017571
121300000 53.19361745325985
121400000 41.64008588590057
121500000 44.04346041365
121600000 40.0147815240536
121700000 51.097935621701644
121800000 51.37377618035215
121900000 51.727956634473
122000000 54.60220129635748
122100000 43.49210883487168
122200000 65.07131741519011
122300000 53.59002527137762
122400000 42.83760364329769
122500000 45.446445386621555
122600000 45.239195944056696
122700000 42.32336944889858
122800000 50.822326366351824
122900000 31.100591227841193
123000000 38.389849188226144
123100000 10.070309038541856
123200000 46.09429173410582
123300000 29.959231011536456
123400000 40.25535491261054
123500000 41.445

...

1189800000 4.8690213710679116
1189900000 3.0357651026749477
1190000000 2.8105415831224128
1190100000 3.464598139776975
1190200000 3.4933591887510156
1190300000 2.9798197425886856
1190400000 3.7482895886938072
1190500000 3.451239554287278
1190600000 3.214408800523036
1190700000 3.4915703947380283
1190800000 3.0579038692615765
1190900000 2.5635612958171765
1191000000 3.688654773771077
1191100000 3.712386179078721
1191200000 3.0371580888083014
1191300000 3.3989158758664626
1191400000 3.8421706812056566
1191500000 3.151477956246432
1191600000 3.961632441935797
best so far: 0
type: [1, 4, 1, 3, 11] 132
cases of this type: 1581228
1191700000 2.6142229275335374
1191800000 1.6791051402997674
1191900000 1.2665567338072061
1192000000 1.4641878356125155
1192100000 1.6180962747173542
1192200000 1.1468236573162576
1192300000 1.302153212458104
1192400000 1.6226418659020565
1192500000 1.3960625584010977
1192600000 1.455891853623546
1192700000 1.9754841756717265
1192800000 1.5053881691679951
119290000

1203600000 3.635559532273382
1203700000 3.8387599834861157
1203800000 3.450959895016498
1203900000 3.8792214664680458
1204000000 3.6661722374586034
1204100000 2.77774642815717
1204200000 4.0892322964736625
1204300000 3.3593895479559666
1204400000 2.805192975300756
1204500000 3.9695676829516584
1204600000 3.157357162502199
1204700000 3.1295023364997636
1204800000 4.142601290957441
1204900000 2.749739095581571
1205000000 3.567726149989002
1205100000 4.097352406939705
1205200000 2.6531269542802374
1205300000 3.1777171692739525
1205400000 3.76212004507792
1205500000 2.550352710367832
1205600000 3.695432576241374
1205700000 3.6839272229351003
1205800000 2.253146220258888
1205900000 3.7998459509847824
1206000000 3.619541499705901
1206100000 2.3957022764897977
1206200000 3.7410902614295085
1206300000 3.8756704843173626
1206400000 2.0856269635550975
1206500000 3.712925967272533
1206600000 3.6364325801383455
1206700000 2.448156552955651
1206800000 4.137291603406151
1206900000 3.4980409285602367

1230800000 0.7838644493167434
1230900000 1.1417185245868366
1231000000 1.2847184166117378
1231100000 0.8918570737433101
1231200000 1.2367978380343319
1231300000 1.216809205054495
1231400000 0.7176696836612126
1231500000 1.1925023999485376
1231600000 1.0547190979643033
1231700000 0.7168039707724552
1231800000 1.2248694669482112
1231900000 1.0672416629030075
1232000000 0.7654854482226437
1232100000 1.2809921353896756
1232200000 0.9467676343133152
1232300000 0.9329372925499478
1232400000 1.1142588487008511
1232500000 0.872625186977042
1232600000 0.8528639927023285
1232700000 1.1318295183464389
1232800000 0.8447681831964851
1232900000 1.0958798558498455
best so far: 0
type: [2, 1, 1, 6, 11] 132
cases of this type: 3162456
1233000000 1.0918685377000272
1233100000 0.7477343514955839
1233200000 0.8781884350897801
1233300000 1.1827009831548037
1233400000 0.6534382493467463
1233500000 0.9373832491288085
1233600000 1.0117438713335594
1233700000 0.5970574536090427
1233800000 1.0832455606461266
12

1252400000 0.49730065139664203
1252500000 0.47700805192171536
1252600000 0.5297820134923855
1252700000 0.5078368824511866
1252800000 0.4555997424082259
1252900000 0.48662459385291407
1253000000 0.45904671749565346
1253100000 0.44959752010136844
1253200000 0.4902503878392345
best so far: 0
type: [2, 2, 1, 3, 11] 132
cases of this type: 790614
1253300000 0.36973679414637217
1253400000 0.2909005310847263
1253500000 0.25146983888386026
1253600000 0.24541717117141354
1253700000 0.26511180513100824
1253800000 0.26224556292676926
1253900000 0.28893573365848596
1254000000 0.1830795276019573
best so far: 0
type: [2, 2, 1, 11, 3] 132
cases of this type: 215622
1254100000 0.283533864143908
1254200000 0.2901062132649819
best so far: 0
type: [2, 2, 1, 33, 1] 132
cases of this type: 71874
1254300000 0.18498389440432192
best so far: 0
type: [2, 2, 3, 1, 11] 132
cases of this type: 263538
1254400000 0.2986132653460171
1254500000 0.16515167218878535
best so far: 0
type: [2, 2, 3, 11, 1] 132
cases of th

best so far: 0
type: [3, 2, 11, 1, 2] 132
cases of this type: 3872
best so far: 0
type: [3, 2, 11, 2, 1] 132
cases of this type: 1936
best so far: 0
type: [3, 2, 22, 1, 1] 132
cases of this type: 968
best so far: 0
type: [3, 4, 1, 1, 11] 132
cases of this type: 58564
best so far: 0
type: [3, 4, 1, 11, 1] 132
cases of this type: 5324
best so far: 0
type: [3, 4, 11, 1, 1] 132
cases of this type: 484
best so far: 0
type: [3, 11, 1, 1, 4] 132
cases of this type: 2816
best so far: 0
type: [3, 11, 1, 2, 2] 132
cases of this type: 1408
best so far: 0
type: [3, 11, 1, 4, 1] 132
cases of this type: 704
best so far: 0
type: [3, 11, 2, 1, 2] 132
cases of this type: 704
best so far: 0
type: [3, 11, 2, 2, 1] 132
cases of this type: 352
best so far: 0
type: [3, 11, 4, 1, 1] 132
cases of this type: 176
best so far: 0
type: [3, 22, 1, 1, 2] 132
cases of this type: 352
best so far: 0
type: [3, 22, 1, 2, 1] 132
cases of this type: 176
best so far: 0
type: [3, 22, 2, 1, 1] 132
cases of this type: 88
best